In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('rheeza').getOrCreate()

In [16]:
from pyspark.sql import functions as fn

## Understand the Dataset

In [35]:
trials_df = spark.read.json('dataset.json', multiLine=True)  #Reading the Json

In [36]:
trials_df.show(5)

+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|ageofparticipant|           clinician|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|              result|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|              19|{Ontario, Saul, t...|  Placebo|    1619827200000|      1617235200000|                            52|{BP normalized, r...|
|              14|{Ontario, Saul, n...| Naproxen|    1619827200000|      1617235200000|                            78|    {Follow-up, N/A}|
|              17|{Ontario, Saul, n...|  Placebo|    1619827200000|      1617235200000|                            14|    {Follow-up, N/A}|
|              18|{Ontario, Will, n...| Naproxen|    1619827200000|      1617235200000|                            14|{BP normalized, N/A}|
|              17|{O

In [37]:
trials_df.printSchema() #Checking the schema

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [38]:
trials_df.columns

['ageofparticipant',
 'clinician',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result']

In [39]:
columns = ['ageofparticipant',
 'clinician.branch',
 'clinician.name',
 'clinician.role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result.conclusion',
 'result.sideeffectsonparticipant']

In [41]:
trials_df.select(columns).show()

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

In [42]:
trials_df.select([fn.count(fn.when(fn.col(column).isNull(), column)).alias(column) for column in columns]).show()

+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|ageofparticipant|clinician.branch|clinician.name|clinician.role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|result.conclusion|result.sideeffectsonparticipant|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+
|               0|               0|             0|           109|        0|                0|                  0|                            73|               53|                              0|
+----------------+----------------+--------------+--------------+---------+-----------------+-------------------+------------------------------+-----------------+-------------------------------+



# Cleaning of Data

In [59]:
new_trials_df = trials_df.select(columns)
new_trials_df.printSchema()


root
 |-- ageofparticipant: long (nullable = true)
 |-- branch: string (nullable = true)
 |-- name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- conclusion: string (nullable = true)
 |-- sideeffectsonparticipant: string (nullable = true)



In [60]:
new_column_names = {'ageofparticipant':'age_of_participant',
 'branch': 'clinic_branch',
 'name': 'head_clinician',
 'role': 'assistant_role',
 'drug_used':'drug_used',
 'experimentenddate':'experiment_date',
 'experimentstartdate':'experiment_start_date',
 'noofhourspassedatfirstreaction':'hours_passed_at_first_reaction',
 'conclusion':'conclusion',
 'sideeffectsonparticipant':'adverse_side_effect'
    
}

In [63]:
#Rename

new_trials_df = new_trials_df.withColumnsRenamed(new_column_names)

In [64]:
new_trials_df.describe().show()

+-------+------------------+-------------+--------------+--------------+---------+--------------------+---------------------+------------------------------+-------------+-------------------+
|summary|age_of_participant|clinic_branch|head_clinician|assistant_role|drug_used|     experiment_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|adverse_side_effect|
+-------+------------------+-------------+--------------+--------------+---------+--------------------+---------------------+------------------------------+-------------+-------------------+
|  count|              3586|         3586|          3586|          3477|     3586|                3586|                 3586|                          3513|         3533|               3586|
|   mean|17.507250418293363|         null|          null|          null|     null|1.618381578137200...| 1.615813671834913...|             44.89097637346997|         null|               null|
| stddev|2.3066401927555233|         null|   

In [66]:
new_trials_df = new_trials_df.na.fill({'conclusion': 'unknown', 'assistant_role': 'unknown'})

In [67]:
new_trials_df.describe().show()

+-------+------------------+-------------+--------------+--------------+---------+--------------------+---------------------+------------------------------+-------------+-------------------+
|summary|age_of_participant|clinic_branch|head_clinician|assistant_role|drug_used|     experiment_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|adverse_side_effect|
+-------+------------------+-------------+--------------+--------------+---------+--------------------+---------------------+------------------------------+-------------+-------------------+
|  count|              3586|         3586|          3586|          3586|     3586|                3586|                 3586|                          3513|         3586|               3586|
|   mean|17.507250418293363|         null|          null|          null|     null|1.618381578137200...| 1.615813671834913...|             44.89097637346997|         null|               null|
| stddev|2.3066401927555233|         null|   